In [3]:
import numpy as np

class node:
    def __init__(self):
        self.hist = np.zeros(3)
        self.prog = np.zeros(3)
        self.turns = -1

    def deal(self):
        self.cards = np.random.choice(3,(2,1))
        self.turns += 1
        return self

    def place(self, action):
        try:
            self.hist[self.turns] = action
            self.prog[self.turns] = 1
            self.turns += 1
        except:
            print(self.prog)
            print(self.hist)
            print(action)
            return None
        return self

    def A(self):
        if self.turns == 0:
            return [1,2]
        elif self.turns == 1:
            if self.hist[0] == 1:
                return [1,2]
            else:
                return [0,2]
        else:
            return [0,2]

    def P(self):
        if self.turns == -1:
            return -1
        else:
            return (self.turns+1)%2

    def is_terminal(self):
        if self.prog[2]:
            return True
        elif not self.prog[1]:
            return False
        elif self.hist[1] != 2:
            return True
        elif self.hist[0] == 2:
            return True
        else:
            return False

    def util(self, p):
        thd = self.hist[2]
        if thd == 0:
            return [0,3][p]
        elif thd == 2:
            return (p == np.argmax(self.cards))*4
        else:
            sec = self.hist[1]
            if sec == 1:
                return (p == np.argmax(self.cards))*2
            elif sec == 2:
                return (p == np.argmax(self.cards))*4
            else:
                return [3,0][p]

    def I(self, p):
        return [self.cards[p], self.hist, self.prog]


In [4]:
h = node()
h.hist=[1,2,0]
h.prog=[1,1,1]
h.is_terminal()

True